# Tutorial 1: Introducción al Análisis Topológico de Datos (TDA)

## Aplicaciones en Neurociencias

**Autor:** MARK-126  
**Nivel:** Principiante-Intermedio  
**Tiempo estimado:** 90-120 minutos

---

## Objetivos de Aprendizaje

Al completar este tutorial, serás capaz de:

1. ✅ Comprender los conceptos fundamentales de topología aplicados a datos
2. ✅ Construir y analizar complejos simpliciales
3. ✅ Calcular homología y números de Betti
4. ✅ Aplicar TDA a datos neurocientíficos simples
5. ✅ Visualizar e interpretar resultados topológicos

---

## 1. ¿Qué es la Topología?

### 1.1 Intuición

La **topología** es la rama de las matemáticas que estudia las propiedades que se preservan bajo deformaciones continuas. 

**Piensa en ello así:**
- Una taza de café y una dona (rosquilla) son topológicamente equivalentes porque puedes deformar una en la otra sin cortar ni pegar
- Ambas tienen **un hueco** (característica topológica fundamental)

### 1.2 ¿Por qué es útil en Neurociencias?

El cerebro es un sistema complejo donde:
- Las **neuronas** forman redes con estructura topológica
- Los **patrones de activación** crean espacios de alta dimensión
- La **conectividad funcional** puede verse como un grafo complejo
- Las características topológicas son **robustas al ruido**

**Ejemplo concreto:** Si dos estados cerebrales tienen la misma topología, esto sugiere que son funcionalmente similares, ¡incluso si las activaciones neuronales específicas difieren!

---

## 2. Setup e Importaciones

In [ ]:
# Importaciones estándar
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
import warnings
warnings.filterwarnings('ignore')

# TDA libraries
from ripser import ripser
from persim import plot_diagrams
import networkx as nx
from scipy.spatial.distance import pdist, squareform
from sklearn.datasets import make_circles, make_moons

# Configuración de visualización
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

# Configuración de reproducibilidad
np.random.seed(42)

print("✅ Todas las bibliotecas importadas correctamente")
print(f"✅ NumPy version: {np.__version__}")

---

## 3. Conceptos Fundamentales

### 3.1 Complejos Simpliciales

Un **complejo simplicial** es una construcción matemática que nos permite representar la "forma" de un conjunto de datos.

**Componentes:**
- **0-simplejo:** Un punto (vértice)
- **1-simplejo:** Una arista (línea entre dos puntos)
- **2-simplejo:** Un triángulo (cara)
- **3-simplejo:** Un tetraedro

**Analogía neuronal:**
- Neuronas = vértices (0-simplejos)
- Conexiones pareadas = aristas (1-simplejos)
- Triadas funcionales = triángulos (2-simplejos)

### 3.2 Construcción: Complejo de Vietoris-Rips

Dado un conjunto de puntos y un radio $\epsilon$:
1. Conecta dos puntos si su distancia es $\leq \epsilon$
2. Si todos los puntos de un conjunto están conectados entre sí, forma un simplejo de dimensión superior

---

## 4. Ejemplo Práctico: Construcción de Complejo Simplicial

In [ ]:
def visualize_simplicial_complex(points, epsilon, title="Complejo Simplicial"):
    """
    Visualiza un complejo simplicial construido con radio epsilon.
    
    Parameters:
    -----------
    points : numpy array
        Puntos en 2D (shape: n_points x 2)
    epsilon : float
        Radio para construcción de Vietoris-Rips
    title : str
        Título del gráfico
    """
    fig, ax = plt.subplots(figsize=(10, 8))
    
    # Grafo para representar el complejo
    G = nx.Graph()
    n_points = len(points)
    
    # Agregar nodos
    for i in range(n_points):
        G.add_node(i, pos=points[i])
    
    # Calcular distancias
    distances = squareform(pdist(points))
    
    # Agregar aristas si distancia <= epsilon
    edges = []
    for i in range(n_points):
        for j in range(i+1, n_points):
            if distances[i, j] <= epsilon:
                G.add_edge(i, j)
                edges.append((i, j))
    
    # Encontrar triángulos (2-simplejos)
    triangles = []
    for i in range(n_points):
        for j in range(i+1, n_points):
            for k in range(j+1, n_points):
                if (distances[i,j] <= epsilon and 
                    distances[j,k] <= epsilon and 
                    distances[i,k] <= epsilon):
                    triangles.append([i, j, k])
    
    # Dibujar triángulos (sombreados)
    for tri in triangles:
        triangle_points = points[tri]
        ax.fill(triangle_points[:, 0], triangle_points[:, 1], 
                alpha=0.2, color='lightblue', edgecolor='none')
    
    # Dibujar aristas
    for i, j in edges:
        ax.plot([points[i, 0], points[j, 0]], 
                [points[i, 1], points[j, 1]], 
                'gray', linewidth=1.5, alpha=0.6, zorder=1)
    
    # Dibujar puntos
    ax.scatter(points[:, 0], points[:, 1], 
               c='red', s=200, zorder=3, edgecolors='black', linewidths=2)
    
    # Etiquetas
    for i, point in enumerate(points):
        ax.annotate(f'{i}', xy=point, xytext=(5, 5), 
                    textcoords='offset points', fontsize=12, fontweight='bold')
    
    ax.set_title(f"{title}\n(ε = {epsilon:.2f}, Aristas: {len(edges)}, Triángulos: {len(triangles)})", 
                 fontsize=14, fontweight='bold')
    ax.set_aspect('equal')
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    return len(edges), len(triangles)

# Ejemplo: Red neuronal simplificada (5 neuronas)
neural_positions = np.array([
    [0, 0],      # Neurona 0
    [1, 0],      # Neurona 1
    [0.5, 0.8],  # Neurona 2
    [2, 0],      # Neurona 3
    [1.5, 0.8]   # Neurona 4
])

print("📊 Visualizando complejos simpliciales con diferentes radios\n")

# Explorar diferentes valores de epsilon
for eps in [0.5, 1.0, 1.5]:
    n_edges, n_triangles = visualize_simplicial_complex(
        neural_positions, eps, 
        title=f"Red Neuronal: Complejo Simplicial"
    )
    print(f"ε = {eps:.1f}: {n_edges} conexiones, {n_triangles} triadas\n")

### 💡 Interpretación Neuronal

- **ε pequeño:** Solo neuronas muy cercanas están conectadas
- **ε grande:** Aparecen más conexiones y triadas funcionales
- Los **triángulos** representan grupos de neuronas que están funcionalmente conectadas entre sí

**Pregunta de reflexión:** ¿Qué valor de ε revela la estructura más interesante?

---

## 5. Homología: Contando Huecos

### 5.1 Teoría

La **homología** es una herramienta algebraica para contar "huecos" en diferentes dimensiones:

- **H₀ (Dimensión 0):** Componentes conectadas
- **H₁ (Dimensión 1):** Ciclos/bucles (como el hueco en una dona)
- **H₂ (Dimensión 2):** Cavidades (como el interior de una esfera)
- **Hₙ (Dimensión n):** Huecos n-dimensionales

### 5.2 Números de Betti

Los **números de Betti** ($\beta_i$) cuentan el número de huecos en cada dimensión:
- $\beta_0$ = número de componentes conectadas
- $\beta_1$ = número de ciclos
- $\beta_2$ = número de cavidades

### 5.3 Aplicación Neural

En una red neuronal:
- $\beta_0$: Número de subredes desconectadas
- $\beta_1$: Circuitos recurrentes (loops de retroalimentación)
- $\beta_2$: Estructuras volumétricas complejas

---

## 6. Ejemplo: Cálculo de Números de Betti

In [ ]:
def compute_betti_numbers(points, max_epsilon=2.0, num_steps=50):
    """
    Calcula números de Betti para diferentes valores de epsilon.
    
    Parameters:
    -----------
    points : numpy array
        Puntos a analizar
    max_epsilon : float
        Máximo radio de análisis
    num_steps : int
        Número de pasos de epsilon
        
    Returns:
    --------
    epsilons : array
        Valores de epsilon
    betti_0, betti_1, betti_2 : arrays
        Números de Betti en cada dimensión
    """
    epsilons = np.linspace(0.01, max_epsilon, num_steps)
    betti_0 = np.zeros(num_steps)
    betti_1 = np.zeros(num_steps)
    betti_2 = np.zeros(num_steps)
    
    # Calcular homología persistente
    result = ripser(points, maxdim=2, thresh=max_epsilon)
    diagrams = result['dgms']
    
    # Para cada epsilon, contar características que existen
    for i, eps in enumerate(epsilons):
        # Dimensión 0 (componentes)
        betti_0[i] = np.sum((diagrams[0][:, 0] <= eps) & 
                           ((diagrams[0][:, 1] > eps) | np.isinf(diagrams[0][:, 1])))
        
        # Dimensión 1 (ciclos)
        if len(diagrams) > 1:
            betti_1[i] = np.sum((diagrams[1][:, 0] <= eps) & 
                               (diagrams[1][:, 1] > eps))
        
        # Dimensión 2 (cavidades)
        if len(diagrams) > 2:
            betti_2[i] = np.sum((diagrams[2][:, 0] <= eps) & 
                               (diagrams[2][:, 1] > eps))
    
    return epsilons, betti_0, betti_1, betti_2

def plot_betti_curves(epsilons, betti_0, betti_1, betti_2, title="Curvas de Betti"):
    """
    Visualiza la evolución de los números de Betti.
    """
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    
    # β₀
    axes[0].plot(epsilons, betti_0, linewidth=3, color='#e74c3c')
    axes[0].fill_between(epsilons, betti_0, alpha=0.3, color='#e74c3c')
    axes[0].set_xlabel('Radio ε', fontsize=12)
    axes[0].set_ylabel('β₀ (Componentes)', fontsize=12)
    axes[0].set_title('Dimensión 0: Componentes Conectadas', fontsize=14, fontweight='bold')
    axes[0].grid(True, alpha=0.3)
    
    # β₁
    axes[1].plot(epsilons, betti_1, linewidth=3, color='#3498db')
    axes[1].fill_between(epsilons, betti_1, alpha=0.3, color='#3498db')
    axes[1].set_xlabel('Radio ε', fontsize=12)
    axes[1].set_ylabel('β₁ (Ciclos)', fontsize=12)
    axes[1].set_title('Dimensión 1: Ciclos/Loops', fontsize=14, fontweight='bold')
    axes[1].grid(True, alpha=0.3)
    
    # β₂
    axes[2].plot(epsilons, betti_2, linewidth=3, color='#2ecc71')
    axes[2].fill_between(epsilons, betti_2, alpha=0.3, color='#2ecc71')
    axes[2].set_xlabel('Radio ε', fontsize=12)
    axes[2].set_ylabel('β₂ (Cavidades)', fontsize=12)
    axes[2].set_title('Dimensión 2: Cavidades', fontsize=14, fontweight='bold')
    axes[2].grid(True, alpha=0.3)
    
    plt.suptitle(title, fontsize=16, fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.show()

# Ejemplo 1: Círculo (debe tener β₁ = 1)
print("📐 Ejemplo 1: Análisis de un círculo\n")
circle_points, _ = make_circles(n_samples=100, noise=0.05, factor=0.5, random_state=42)
circle_points = circle_points[circle_points[:, 0]**2 + circle_points[:, 1]**2 > 0.1]  # Solo círculo exterior

eps, b0, b1, b2 = compute_betti_numbers(circle_points, max_epsilon=1.0)
plot_betti_curves(eps, b0, b1, b2, title="Topología de un Círculo")

print("✅ Observa cómo β₁ = 1 (un ciclo) persiste en un rango de ε")
print("✅ β₀ comienza alto (muchos componentes) y converge a 1\n")

### 🧠 Interpretación Neuronal

Si aplicamos esto a una red neuronal:
- **β₀ alto inicial:** Muchas neuronas individuales sin conexiones
- **β₀ → 1:** La red se conecta completamente
- **β₁ > 0:** Aparecen circuitos recurrentes (fundamentales para memoria y aprendizaje)

---

## 7. Homología Persistente

### 7.1 ¿Por qué "Persistente"?

No todas las características topológicas son igualmente importantes:
- Algunas aparecen por **ruido** (nacen y mueren rápidamente)
- Otras son **robustas** (persisten a través de múltiples escalas)

**Homología Persistente** rastrea cuándo nacen y mueren las características a medida que aumentamos ε.

### 7.2 Diagrama de Persistencia

Un **diagrama de persistencia** es un scatter plot donde:
- **Eje X:** Nacimiento (birth) - valor de ε donde aparece la característica
- **Eje Y:** Muerte (death) - valor de ε donde desaparece
- **Distancia a la diagonal:** Persistencia (lifetime)

**Regla de oro:** Puntos lejos de la diagonal = características significativas

---

## 8. Ejemplo Completo: Datos Neuronales Sintéticos

In [ ]:
def generate_neural_network(n_neurons=50, connectivity=0.3, noise_level=0.1):
    """
    Genera una red neuronal sintética con estructura de comunidades.
    
    Parameters:
    -----------
    n_neurons : int
        Número de neuronas
    connectivity : float
        Nivel de conectividad (0-1)
    noise_level : float
        Cantidad de ruido
    """
    # Crear dos comunidades de neuronas
    community1 = np.random.randn(n_neurons//2, 2) * 0.5 + np.array([0, 0])
    community2 = np.random.randn(n_neurons//2, 2) * 0.5 + np.array([3, 0])
    
    # Agregar una neurona puente
    bridge = np.array([[1.5, 0]])
    
    # Combinar
    neurons = np.vstack([community1, community2, bridge])
    
    # Agregar ruido
    neurons += np.random.randn(*neurons.shape) * noise_level
    
    return neurons

# Generar red neuronal
print("🧠 Generando red neuronal sintética con dos comunidades...\n")
neural_network = generate_neural_network(n_neurons=60, connectivity=0.3, noise_level=0.08)

# Visualizar
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Plot 1: Red neuronal
axes[0].scatter(neural_network[:, 0], neural_network[:, 1], 
                c='blue', s=100, alpha=0.6, edgecolors='black')
axes[0].set_title('Red Neuronal Sintética\n(2 comunidades + neurona puente)', 
                  fontsize=14, fontweight='bold')
axes[0].set_xlabel('Dimensión 1', fontsize=12)
axes[0].set_ylabel('Dimensión 2', fontsize=12)
axes[0].grid(True, alpha=0.3)
axes[0].set_aspect('equal')

# Plot 2: Diagrama de persistencia
print("⏳ Calculando homología persistente...")
result = ripser(neural_network, maxdim=2)
diagrams = result['dgms']

plot_diagrams(diagrams, ax=axes[1])
axes[1].set_title('Diagrama de Persistencia', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Nacimiento (Birth)', fontsize=12)
axes[1].set_ylabel('Muerte (Death)', fontsize=12)

plt.tight_layout()
plt.show()

print("\n✅ Análisis completado!")
print(f"\n📊 Resultados:")
print(f"   • H₀: {len(diagrams[0])} componentes detectadas")
print(f"   • H₁: {len(diagrams[1])} ciclos detectados")
if len(diagrams) > 2:
    print(f"   • H₂: {len(diagrams[2])} cavidades detectadas")

### 🔍 Interpretación del Diagrama

**Dimensión 0 (rojo):**
- Muchos puntos cerca del eje Y → neuronas individuales que se fusionan rápidamente
- Un punto muy persistente → la red completa eventualmente se conecta

**Dimensión 1 (azul):**
- Puntos lejos de la diagonal → ciclos robustos en la red
- Estos representan circuitos de retroalimentación neuronal

**Dimensión 2 (verde):**
- Cavidades volumétricas en la estructura de la red

---

## 9. Aplicación: Comparación de Estados Cerebrales

In [ ]:
def generate_brain_state(state_type='resting', n_neurons=100):
    """
    Genera patrones de activación neuronal para diferentes estados cerebrales.
    
    Parameters:
    -----------
    state_type : str
        'resting' o 'active'
    n_neurons : int
        Número de neuronas
    """
    if state_type == 'resting':
        # Estado de reposo: activación dispersa
        data = np.random.randn(n_neurons, 3) * 1.5
    elif state_type == 'active':
        # Estado activo: estructura más organizada (esfera)
        theta = np.random.uniform(0, 2*np.pi, n_neurons)
        phi = np.random.uniform(0, np.pi, n_neurons)
        r = 1 + np.random.randn(n_neurons) * 0.1
        
        x = r * np.sin(phi) * np.cos(theta)
        y = r * np.sin(phi) * np.sin(theta)
        z = r * np.cos(phi)
        
        data = np.column_stack([x, y, z])
    
    return data

# Generar estados
print("🧠 Generando estados cerebrales sintéticos...\n")
resting_state = generate_brain_state('resting', n_neurons=150)
active_state = generate_brain_state('active', n_neurons=150)

# Calcular homología para ambos estados
print("⏳ Analizando topología...")
result_resting = ripser(resting_state, maxdim=2, thresh=3.0)
result_active = ripser(active_state, maxdim=2, thresh=3.0)

# Visualizar
fig = plt.figure(figsize=(18, 6))

# Estado de reposo - 3D
ax1 = fig.add_subplot(131, projection='3d')
ax1.scatter(resting_state[:, 0], resting_state[:, 1], resting_state[:, 2],
            c='blue', alpha=0.4, s=30)
ax1.set_title('Estado de Reposo\n(activación dispersa)', fontsize=12, fontweight='bold')
ax1.set_xlabel('X')
ax1.set_ylabel('Y')
ax1.set_zlabel('Z')

# Estado activo - 3D
ax2 = fig.add_subplot(132, projection='3d')
ax2.scatter(active_state[:, 0], active_state[:, 1], active_state[:, 2],
            c='red', alpha=0.4, s=30)
ax2.set_title('Estado Activo\n(estructura esférica)', fontsize=12, fontweight='bold')
ax2.set_xlabel('X')
ax2.set_ylabel('Y')
ax2.set_zlabel('Z')

# Comparación de persistencia
ax3 = fig.add_subplot(133)
plot_diagrams([result_resting['dgms'][1]], ax=ax3, legend=False)
ax3.scatter(result_active['dgms'][1][:, 0], result_active['dgms'][1][:, 1],
            marker='s', c='red', s=50, alpha=0.6, label='Estado Activo')
ax3.set_title('Comparación: H₁ (Ciclos)', fontsize=12, fontweight='bold')
ax3.legend(['Reposo', 'Activo'])

plt.tight_layout()
plt.show()

# Análisis cuantitativo
print("\n📊 Análisis Comparativo:")
print("\n🔵 Estado de Reposo:")
print(f"   • H₁ (ciclos): {len(result_resting['dgms'][1])}")
print(f"   • H₂ (cavidades): {len(result_resting['dgms'][2])}")

print("\n🔴 Estado Activo:")
print(f"   • H₁ (ciclos): {len(result_active['dgms'][1])}")
print(f"   • H₂ (cavidades): {len(result_active['dgms'][2])}")

print("\n💡 Interpretación:")
print("   El estado activo muestra estructura esférica → H₂ > 0 (cavidad)")
print("   El estado de reposo es más disperso → menos estructura topológica")
print("   ¡TDA revela diferencias cualitativas entre estados cerebrales!")

---

## 10. Ejercicios Prácticos

### Ejercicio 1: Análisis de tu propia red

Genera una red neuronal con diferentes parámetros y analiza su topología:

```python
# Tu código aquí
mi_red = generate_neural_network(n_neurons=80, connectivity=0.5, noise_level=0.15)
# Calcula y visualiza la homología persistente
```

**Preguntas:**
1. ¿Cuántos ciclos robustos detectas?
2. ¿Cómo cambia la topología con diferentes niveles de ruido?
3. ¿Qué ocurre si aumentas la conectividad?

---

In [ ]:
# Espacio para Ejercicio 1


### Ejercicio 2: Comparación de formas

Compara la topología de diferentes formas geométricas:

```python
from sklearn.datasets import make_circles, make_moons, make_blobs

# Genera diferentes datasets
circles, _ = make_circles(n_samples=100, noise=0.05, factor=0.5)
moons, _ = make_moons(n_samples=100, noise=0.05)
blobs, _ = make_blobs(n_samples=100, centers=3, cluster_std=0.5)

# Analiza cada uno con TDA
```

**Preguntas:**
1. ¿Qué forma tiene β₁ más alto?
2. ¿Puedes distinguir las formas solo mirando sus diagramas de persistencia?

---

In [ ]:
# Espacio para Ejercicio 2


### Ejercicio 3: Sensibilidad al ruido

Investiga cómo el ruido afecta la detección de características topológicas:

```python
# Genera un círculo limpio
clean_circle, _ = make_circles(n_samples=100, noise=0.01, factor=0.5)

# Agregar diferentes niveles de ruido y comparar
for noise_level in [0.05, 0.1, 0.2]:
    noisy = clean_circle + np.random.randn(*clean_circle.shape) * noise_level
    # Analizar topología
```

**Pregunta:** ¿Cuánto ruido puede tolerar antes de que desaparezca el ciclo principal?

---

In [ ]:
# Espacio para Ejercicio 3


## 11. Resumen y Puntos Clave

### ✅ Lo que aprendimos:

1. **TDA** estudia la "forma" de los datos mediante conceptos topológicos
2. **Complejos simpliciales** representan estructura multi-escala
3. **Homología** cuenta huecos en diferentes dimensiones
4. **Persistencia** distingue características robustas del ruido
5. **Aplicaciones neurales**: redes, conectividad, estados cerebrales

### 🔑 Conceptos Clave:

- **β₀:** Componentes conectadas (subredes)
- **β₁:** Ciclos (circuitos de retroalimentación)
- **β₂:** Cavidades (estructuras volumétricas)
- **Diagrama de persistencia:** Mapa de vida de características
- **Robustez:** TDA es invariante a deformaciones continuas

### 🧠 Intuición Neural:

El cerebro no es solo un conjunto de neuronas conectadas, sino una estructura **topológica compleja** donde:
- Los ciclos representan memoria y retroalimentación
- Las cavidades indican organización jerárquica
- La persistencia revela patrones funcionalmente significativos

---

## 12. Próximos Pasos

En los siguientes tutoriales exploraremos:

### Tutorial 2: Homología Persistente Avanzada
- Cálculo eficiente con grandes datasets
- Filtración Rips vs otros métodos
- Distancias entre diagramas (Wasserstein, Bottleneck)
- Aplicación a patrones de spike trains

### Tutorial 3: Conectividad Cerebral
- Análisis de matrices de correlación fMRI
- Detección de comunidades topológicas
- Estudio de redes funcionales vs estructurales

### Tutorial 4: Algoritmo Mapper
- Visualización de espacios de alta dimensión
- Descubrimiento de subespacios neuronales
- Análisis de trayectorias cerebrales

### Tutorial 5: Series Temporales
- Embeddings de Takens para señales EEG
- Detección de eventos mediante TDA
- Clasificación de estados cognitivos

---

## 13. Referencias y Recursos

### Papers Fundamentales:
1. Carlsson, G. (2009). "Topology and data". *Bulletin of the American Mathematical Society*
2. Giusti, C., et al. (2015). "Clique topology reveals intrinsic structure in neural correlations". *PNAS*
3. Curto, C. (2017). "What can topology tell us about the neural code?". *Bulletin of the AMS*

### Libros:
- Edelsbrunner & Harer (2010). *Computational Topology: An Introduction*
- Ghrist, R. (2014). *Elementary Applied Topology*

### Software:
- [Ripser](https://ripser.scikit-tda.org/) - Cálculo rápido de homología
- [Giotto-TDA](https://giotto-ai.github.io/gtda-docs/) - Suite completa de TDA
- [GUDHI](https://gudhi.inria.fr/) - Biblioteca robusta

### Cursos Online:
- Applied Algebraic Topology (Stanford)
- Topological Data Analysis (Coursera)

---

## 🎉 ¡Felicitaciones!

Has completado la introducción al Análisis Topológico de Datos aplicado a neurociencias. Ahora tienes las herramientas fundamentales para explorar la "forma" de datos neuronales complejos.

**¿Preguntas? ¿Ideas para explorar?** Abre un issue en el repositorio o continúa con el siguiente tutorial.

---

**Autor:** MARK-126  
**Última actualización:** 2025-01-13  
**Licencia:** MIT